# Gravity export

In [34]:
import pandas as pd
import json

In [35]:
d = []
with open("gravity_res_big_formatted.json") as f:
    for line in f:
        d.append(json.loads(line.strip("\n")))

In [36]:
df = pd.DataFrame.from_dict(d)

In [37]:
def data_series(d,t):
    temp = []
    for k in d:
        temp.append((float(k),d[k]['1']/d[k]['0']))
    return pd.DataFrame(temp).rename({1:t},axis=1).set_index(0)

In [38]:
for key in ["month","ABM1","ABM2"]:
    pd.concat(df[df.data_source == key]\
        .apply(lambda row: data_series(row["gravity_dict"],row["category"]),axis=1)\
        .to_list(),axis=1)\
        .fillna(0)\
        .reset_index()\
        .rename({0:"d_km"},axis=1)\
        .to_csv("gravity_plot_"+key+".csv")

# BFS

In [1]:
import pandas as pd
import numpy as np
import json
from itertools import product
from time import time

from SI import SI, DE_fit

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.optimize import curve_fit
from scipy.stats import pearsonr

from IPython.display import display, Markdown

In [2]:
plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = 10,7

In [3]:
# ABM sample nodes with their empirical adoption times and city ids
nodelist = pd.read_csv('whole_network/nodes_from_edgelist_unique_w_month.csv')
original_timeline = np.concatenate([nodelist.groupby("month").count()["id"].values,[0,0]])
# storing original data in a similar fashion as a simulation output
original_data = {
    "avg_timeline": original_timeline,
    "p": 1e-4,
    "q": 0.1,
    "stop": 129,
    "num_nodes": len(nodelist.index),
    "high": None,
    "low": None
}

In [4]:
sim = SI(**original_data)

In [6]:
np.mean(sim.A.sum(axis=1))

91.70550974366508

In [18]:
clust_coeffs = [sim.A[sim.A[i,:].nonzero()[1],:].tocsc()[:,sim.A[i,:].nonzero()[1]].sum()/(sim.A[0,:].sum()*(sim.A[0,:].sum()-1)) for i in range(sim.A.shape[0])]

MemoryError: 